In [10]:
import collections
import itertools
import random
import graphviz
from sympy import *
import random

In [11]:
def Shortest_Paths_All_Source(Graph):
    temp={}

    for node in Graph:
      temp.update(Shortest_Paths_Single_Source(Graph, node))
    return temp
def Shortest_Paths_Single_Source(Graph, T):
  temp={(T, node): [] for node in Graph}
  UnVisited = set(Graph)
  queue = collections.deque([T])
  while queue:
    curr = queue.popleft()
    if curr not in UnVisited: continue
    for near in Graph[curr]:
      if near in UnVisited:
        path = temp[(T, curr)]
        dist = len(path)
        near_dist = len(temp[(T, near)])
        if dist+1 < near_dist or near_dist == 0:
          temp[(T, near)] = path + [near]
          queue.append(near)
    UnVisited.remove(curr)
  for v in UnVisited:
    del temp[(T, v)]
  return temp

In [12]:
def Stenier_Base_Case(dij):
    b={}
    for (v0, v1), p in dij.items():
        b[frozenset([v0, v1])] = Path_To_Edges(v0, p)
    return b
def Path_To_Edges(v, p):
    edges =[frozenset([e0,e1]) for e0, e1 in zip([v] + p, p)]
    return frozenset(edges)


def Make_Steiner(Graph, Multi_Cst_Node):
  sample=random.sample(Multi_Cst_Node, 1)[0]
  Nodes=Multi_Cst_Node - {sample}
  Shrt_Path=Shortest_Paths_All_Source(Graph)
  Base_Case=Stenier_Base_Case(Shrt_Path)
  temp={}
  for nd in range(2,len(Multi_Cst_Node)):
      for tple in itertools.combinations(Shrt_Path,nd):
          D = frozenset(tple)
          for gnodes in Graph:
              EF_trees=[]
              for first,second in KSubsets(tple,2):
                  EF_trees.append((Base_Case[frozenset([gnodes]+first)],Base_Case[frozenset([gnodes]+second)]))
              temp[(gnodes, D)]=min(EF_trees, key=lambda x: len(x[0])+len(x[1]))
          for nde in {sample} if D == Shrt_Path else Graph:
              Length_Tree = []
              for gnodes in Graph:
                  Length_Tree.append((len(Shrt_Path[(nde,gnodes)])+len(temp[(gnodes,D)][0])+len(temp[(gnodes,D)][1]),gnodes,Path_To_Edges(nde, Shrt_Path[(nde,gnodes)]).union(temp[(gnodes, D)][0]).union(temp[(gnodes, D)][1])))
              length, gnodes,tree = min(Length_Tree,key=lambda x:x[0])
              Base_Case[frozenset(D.union(frozenset([nde])))]=tree
  return Base_Case[frozenset(Multi_Cst_Node)]

In [13]:
#Return Particular Style Edges(Terminal/steiner)
def Grpah_Edges(Graph):
  edges = set()
  for node, es in Graph.items():
    for e in es:
      edges.add(frozenset((node, e)))
  return edges

#Function to Visualize the Graph
def Steiner_Visualize(Graph,terminals,Tedges):
  terminal_style = {'color': 'blue'}
  steiner_style = {'color': 'red'}
  def Steiner_Node(terminals,edges):
    nodes=set()
    for e0,e1 in  edges:
      nodes.add(e0)
      nodes.add(e1)
    return nodes-set(terminals)
  #Return the type of Node
  def Node_Style(node,terminals,stnr_nodes):
    if node in terminals:
      return terminal_style
    elif node in stnr_nodes:
      return steiner_style
    return {}
  #Return Edge of steiner style
  def Edge_Style(e,tredges):
    if e in tredges:
      return steiner_style
    return {}
  #Final Graph Visualization Code
  dot = graphviz.Graph()
  steiner_nodes = Steiner_Node(terminals, Tedges)
  for v, es in Graph.items():
    dot.node(v, **Node_Style(v,terminals,steiner_nodes))
  for v0, v1 in Grpah_Edges(Graph):
    dot.edge(v0, v1, **Edge_Style(frozenset((v0,v1)),Tedges))
  dot.render('Final_Graph.gv')

In [14]:
#Code For K subsets
def KSubsets(ns, m):
  def visit(n, a):
    ps = [[] for i in range(m)]
    for j in range(n):
      ps[a[j + 1]].append(ns[j])
    return ps

  def First(X,Y,rate,n,a):
    if X == 2:
      yield visit(n, a)
    else:
      for v in First(X- 1, Y-1,(X+rate) %2,n,a):
        yield v
      if Y == X + 1:
        a[X] = X - 1
        yield visit(n,a)
        while a[Y] > 0:
          a[Y] = a[Y] - 1
          yield visit(n,a)
      elif Y > X + 1:
        if (X+rate) % 2==1:
          a[Y - 1] = X - 1
        else:
          a[X] = X - 1
          if (a[Y]+rate) % 2 == 1:
            for v in Second(X, Y - 1, 0, n, a):
              yield v
          else:
            for v in First(X, Y - 1, 0, n, a):
              yield v
        while a[Y] > 0:
          a[Y] = a[Y] - 1
          if (a[Y]+rate) % 2 == 1:
            for v in Second(X,Y-1,0,n,a):
              yield v
          else:
            for v in First(X,Y-1,0,n,a):
              yield v

  def Second(X,Y,rate,n,a):
    if Y==X+1:
      while a[Y]<X-1:
        yield visit(n,a)
        a[Y] = a[Y] + 1
      yield visit(n, a)
      a[X] = 0
    elif Y > X + 1:
      if (a[Y]+rate) % 2 == 1:
        for v in First(X,Y-1,0,n,a):
          yield v
      else:
        for v in Second(X,Y-1,0,n,a):
          yield v
      while a[Y]<X-1:
        a[Y]=a[Y]+1
        if (a[Y]+rate)%2== 1:
          for v in First(X,Y-1,0,n,a):
            yield v
        else:
          for v in Second(X,Y-1,0,n,a):
            yield v
      if (X+rate)%2==1:
        a[Y - 1]= 0
      else:
        a[X]=0
    if X == 2:
      yield visit(n,a)
    else:
      for v in Second(X-1,Y-1,(X+rate)%2,n,a):
        yield v

    n=len(ns)
    a=[0] * (n + 1)
    for j in range(1,m+1):
        a[n-m+j]=j-1
    return First(m,n,0,n,a)


In [15]:
def Create_Graph():
  print("Enter the No of vertices in the graph")
  No_Vertices=int(input())
  print("Enter the No of Total Edges")
  Total_Edges=int(input())
  Vertice=[str(i+1) for i in range(No_Vertices)]
  Graph={j: [] for j in Vertice}
  print('Enter the Edges')
  for i in range(1,Total_Edges+1):
    v0=(input())
    v1=(input())
    Graph[v0].append(v1)
    Graph[v1].append(v0)
  print(Graph)
  return Graph

def Multicast_Nodes():
  print("Enter the no of node in multicast network")
  N=int(input())
  print("Enter all the nodes")
  multi_cst_node=[]
  for i in range(N):
    node=input()
    multi_cst_node.append(node)
  print('Enter the Source Node')
  src_node=input()
  return multi_cst_node
#Graph=Create_Graph()
Graph={'1': ['4', '3'], '2': ['5'], '3': ['1','4'],'4': ['1','3','5'],'5': ['2','4']}
nodes=len(Graph)
#multi_cst_node=Multicast_Nodes()
multi_cst_node=['4','2']
terminals=frozenset(multi_cst_node)
tree=Make_Steiner(Graph,terminals)
Steiner_Visualize(Graph,terminals,tree)
print(tree)

frozenset({frozenset({'4', '5'}), frozenset({'5', '2'})})


In [16]:
#To Get All the Edges
final=[]
for Edge in tree:
  temp=[]
  for i in Edge:
    temp.append(i)
  final.append(temp)
print(final)

[['4', '5'], ['5', '2']]


In [17]:
#Getting the Adjancey List for Prufere's Code
adj=[[] for i in range(nodes+1)]
p=[]
for temp in final:
  adj[int(temp[0])].append(int(temp[1]))
  adj[int(temp[1])].append(int(temp[0]))
print(adj)


[[], [], [5], [], [5], [4, 2]]


In [20]:
#Code For Pruferes Algorithm 
parent=[None]*(nodes+1)
Code=[]
def RegParent(v):
  if(v==nodes):
    for i in range(len(adj)):
      if(len(adj[i])>1):
        for j in range(len(adj[i])):
          Code.append(i)
    return -1
  for i in adj[(v)]:
    if(i!=parent[(v)]):
      parent[(i)]=v
      RegParent(i)


def Prufere_Sequence():
  parent[nodes]=-1
  if(RegParent((nodes))==-1):
    return
  ptr=-1
  degree=[0]*(nodes+1)
  for i in range(1,nodes+1):
    degree[i]=len(adj[i])
    if degree[i]==1 and ptr==-1:
      ptr=(i)
  code=[None]*(nodes-2)
  leaf=ptr
  for i in range(nodes-3):
    next =parent[(leaf)]
    code[i]=next
    if (degree[(next)]-1) == 1 and next<ptr:
      leaf=next 
    else:
      ptr=((ptr)+1)
      while ptr<=nodes and degree[(ptr)]!=1  :
        ptr=((ptr)+1)
      leaf=ptr
Prufere_Sequence()
print("Pruefere's Code is :")
print(Code)

Pruefere's Code is :
[5, 5]


In [21]:

def Unique(temp):
	X=[]
	for i in temp:
		if i not in X:
			X.append(i
            )
	if len(X)==len(temp):
		return 1
	else:
		return 0

#Group Deffie Hellman for Multicast Node
def GDH():
  for item in final:  
    q=int(input('Enter prime value for q: '))
    while isprime(q)==0:
      q=int(input("Please enter a PRIME NUMBER ONLY for q: "))
    prim = []
    for first in range(1,q):
      temp1=[]
      for second in range(1,q):
        temp = (first**second)%q
        temp1.append(temp)
      if Unique(temp1):
        prim.append(first)
    print("Primitive roots are "+str(prim))
    alpha = random.choice(prim)
    print('Selected alpha is : '+ str(alpha))
    p = 2
    priv = []
    for i in range(p):
      temp = int(input("Enter "+item[i]+" nd/th node private key: "))
      while temp >= q:
        temp = int(input("Enter "+item[1]+"nd/th node  private key STRICTLY LESS THAN q: "))
      priv.append(temp)
    public=[]
    for i in priv:
      public.append((alpha**i)%q)
    print("Public keys are : "+str(public))
    print("Key exchange for "+item[0] +"nd/th Node... ")
    key1=(public[0]**priv[1])%q
    print(key1)
    print("Key exchange for "+item[1]+" nd/th Node... ")
    key2=(public[1]**priv[0])%q
    print(key2)
    if key1==key2:
      print("Key exchanges are same.")
    else:
      print("Error")
GDH()


Enter prime value for q: 11
Primitive roots are [2, 6, 7, 8]
Selected alpha is : 8
Enter 4 nd/th node private key: 12
Enter 5nd/th node  private key STRICTLY LESS THAN q: 5
Enter 5 nd/th node private key: 6
Public keys are : [10, 3]
Key exchange for 4nd/th Node... 
1
Key exchange for 5 nd/th Node... 
1
Key exchanges are same.
Enter prime value for q: 8
Please enter a PRIME NUMBER ONLY for q: 13
Primitive roots are [2, 6, 7, 11]
Selected alpha is : 2
Enter 5 nd/th node private key: 6
Enter 2 nd/th node private key: 4
Public keys are : [12, 3]
Key exchange for 5nd/th Node... 
1
Key exchange for 2 nd/th Node... 
1
Key exchanges are same.


In [22]:
#Code For BlockChain Induced Node Multiucast Node Connection 
from hashlib import sha256
import json
import time

#Node Structure
class Nodes:
  def __init__(self,ID):
    Chain=[]
    self.Node_Id=ID
    self.Secret_Key=self.Node_Hash=sha256(self.Node_Id.encode()).hexdigest()   
  def SecKey(self):
    return self.Secret_Key
  def NodeLink(self,SecretKey,PrevKey,NextKey):
    self.PrevHash=sha256((str(SecretKey)+str(PrevKey)).encode()).hexdigest()
    self.NextHAsh=sha256((str(SecretKey)+str(NextKey)).encode()).hexdigest()
  def PreKey(self):
    return self.PrevHash
  def nextKey(self):
    return self.NextHAsh

Temp=set()
chain=[]
for  X in final:
  if(X[0] not in Temp):
    node=Nodes(X[0])
    Temp.add(X[0])
    chain.append(node)
  if(X[1] not in Temp):
    node=Nodes(X[1])
    Temp.add(X[1])
    chain.append(node)

prevnode='0'
n=len(chain)
for i in range(0,len(chain)-1):
  chain[i].NodeLink(chain[i].SecKey,prevnode,chain[i+1].SecKey)
  prevnode=chain[i].SecKey

chain[n-1].NodeLink(chain[n-1].SecKey,prevnode,'0')


print("Chain Length is")
print(len(chain))

Chain Length is
3
